# Evaluate building completeness

In support of an ongoing project in Nigeria, we are investigating and evaluating the results of a building digitization effort. The following analyses will be performed to assess the completeness of the digitization effort.

1. Attribute evaluation  
   a. ensure variables in verification protocol are present  
   b. ensure relationship between parcel owner and building occupants  
2. Coverage evaluation  
   a. Calculate across the area in a consistently sized grid (250m)  
   b. Compare building density to Google Buildings  

In [1]:
import sys, os, importlib
import folium

import pandas as pd
import geopandas as gpd
import GOSTRocks.rasterMisc as rMisc
import GOSTRocks.misc as misc

from shapely.geometry import Point, box
from shapely.wkt import loads

from math import ceil
import numpy as np
from shapely.geometry import Polygon

import bldg_helper as helper

%load_ext autoreload
%autoreload 2

In [2]:
in_folder =  "/home/wb411133/projects/NGA_buildings/Jigawa"
out_folder = os.path.join(in_folder, "Results")
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

#Define inputs
in_da = os.path.join(os.path.dirname(in_folder), "NGA_buildings_merge.shp")
in_aoi = os.path.join(os.path.dirname(in_folder), 'NGA_buildings_aois.shp')
in_buildings = os.path.join(in_folder, "Collected","SFTAS_ DLI 11.3_ Jigawa State", "buildings.shp")
in_parcels = os.path.join(in_folder, "Collected","SFTAS_ DLI 11.3_ Jigawa State", "parcels.shp")
    
# Define paramters
crs = 3857
m_crs = f'epsg:{crs}' # projection used to calculate metre measurements
res = 250 # resolution of 

# Define outputs
summary_grid    = os.path.join(out_folder, f"Jigawa_summary_grid_{res}.shp")
out_google_csv  = os.path.join(out_folder, "google_buildings.csv")
out_google_bldg = os.path.join(out_folder, "google_buildings.geojson")
da_buildings    = os.path.join(out_folder, "da_buildings.shp")

In [3]:
inAOI = gpd.read_file(in_aoi)
inAOI = inAOI.loc[inAOI['Name'].apply(lambda x: "Jigawa" in x)]
inAOI

,id,Name,geometry
7,NaN,Jigawa1,"POLYGON ((8.42145 12.66358, 8.43155 12.66263, ..."
8,NaN,Jigawa2,"POLYGON ((9.38901 12.64222, 9.40080 12.64097, ..."
9,NaN,Jigawa3,"POLYGON ((10.03049 12.44692, 10.01655 12.46196..."
10,NaN,Jigawa4,"POLYGON ((9.14631 12.15787, 9.17032 12.15888, ..."
11,NaN,Jigawa5,"POLYGON ((9.31631 11.69089, 9.32957 11.71386, ..."
12,NaN,Jigawa6,"POLYGON ((9.48082 11.46761, 9.48551 11.46653, ..."


In [31]:
inB = gpd.read_file(in_buildings)
inP = gpd.read_file(in_parcels)

if inB.crs.to_epsg() != crs:
    inB = inB.to_crs(m_crs)
    
if inP.crs.to_epsg() != crs:
    inP = inP.to_crs(m_crs)

print(inB.shape)    
inB = inB.loc[inB['geometry'].apply(lambda x: not x is None)]
print(inB.shape)
print("Parcels")
print(inP.shape)
inP = inP.loc[inP['geometry'].apply(lambda x: not x is None)]
print(inP.shape)

inB['geometry'] = inB['geometry'].apply(lambda x: x.buffer(0))
inP['geometry'] = inP['geometry'].apply(lambda x: x.buffer(0))

(27354, 1)
(27244, 1)
Parcels
(23006, 41)
(21877, 41)


In [32]:
inB['area'] = inB['geometry'].apply(lambda x: x.area)
inP['area'] = inP['geometry'].apply(lambda x: x.area)

In [33]:
inP.head()

,fid,first_name,last_name,email,gender,approximat,main_build,enumerator,marital_st,middle_nam,...,file_no,layout_id,created_by,created_at,updated_by,updated_at,layer,path,geometry,area
0,31.0,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,"POLYGON ((1039423.947 1310587.750, 1039476.874...",1794.623557
1,32.0,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,"POLYGON ((1039466.734 1310621.958, 1039500.607...",432.565287
2,33.0,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,"POLYGON ((1039479.659 1310598.558, 1039489.910...",419.739985
3,34.0,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,"POLYGON ((1055681.508 1282951.083, 1055699.032...",270.791953
4,35.0,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,"POLYGON ((1055666.561 1282948.285, 1055681.508...",505.424100


# Calculate general statistics

In [8]:
print(f'Total number of buildings: {inB.shape[0]}')
print(f'Total area of buildings: {inB["area"].sum()}')
      
print(f'Total number of parcels: {inP.shape[0]}')
print(f'Total area of parcels: {inP["area"].sum()}')

Total number of buildings: 27244
Total area of buildings: 2270814.146868458
Total number of parcels: 21877
Total area of parcels: 5805865.088885899


# Extract buildings from Digitize Africa


In [24]:
importlib.reload(helper)
if not os.path.exists(da_buildings):
    inDA = helper.extract_da_buildings(inAOI, in_da, m_crs)
    inDA.to_file(da_buildings)
else:
    inDA = gpd.read_file(da_buildings)

# Assess attribute completeness
1. Size of the parcel (based on polygon)
2. **Number of structures in parcel, number of occupancy units**  
   a. This is not found in the parcel datasets, but can be calculated.
3. The use of each structure and unit
4. The name, ID, and contact details for the owner of the parcel, and the owner or occupier of the structure, or unit


In [ ]:
# Calculate null values in buildings dataset
print(f'Total Records: {inB.shape[0]}')
for col in inB.columns:
    cur_col = inB.loc[:,col]
    print(f'{col}: {cur_col.isna().sum()}')

In [ ]:
# Calculate null values in parcels dataset
print(f'Total Records: {inP.shape[0]}')
for col in inP.columns:
    cur_col = inP.loc[:,col]
    print(f'{col}: {cur_col.isna().sum()}')

# Summarize buildings within a grid

Create a 250m grid across the study arera and summarize bulidngs and parcels within

In [34]:
# Generate the grid
if not os.path.exists(summary_grid):
    grid = helper.generate_grid(inAOI, res, m_crs)
    grid.to_file(summary_grid)
else:
    grid = gpd.read_file(summary_grid)

In [35]:
if inB.crs.to_epsg() != grid.crs.to_epsg():
    inB = inB.to_crs(grid.crs)
    
grid = helper.summarize_in_grid(grid, inDA, inB, inP)

/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError a

/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError a

/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError a

/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError a

/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError a

...

/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError a

/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError a

/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError a

/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError a

/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
  raise InvalidGeometryError("Null geometry supports no operations")
/home/wb411133/.conda/envs/geog/lib/python3.9/site-packages/shapely/topology.py:19: ShapelyDeprecationWarning: InvalidGeometryError will derive from ShapelyError a

In [37]:
grid['per_b'].max()

0.3152617778206195

In [36]:
grid.head()

,index,rowIdx,colIdx,geometry,per_b,per_da,per_p
0,0,934507.424344,1.417413e+06,"POLYGON ((934507.424 1417412.915, 934507.424 1...",0.0,0.000000,0.0
1,1,934507.424344,1.417663e+06,"POLYGON ((934507.424 1417662.915, 934507.424 1...",0.0,0.000000,0.0
2,2,934507.424344,1.417913e+06,"POLYGON ((934507.424 1417912.915, 934507.424 1...",0.0,0.006678,0.0
3,3,934507.424344,1.418163e+06,"POLYGON ((934507.424 1418162.915, 934507.424 1...",0.0,0.010190,0.0
4,4,934507.424344,1.418413e+06,"POLYGON ((934507.424 1418412.915, 934507.424 1...",0.0,0.006803,0.0


In [38]:
grid.to_file(summary_grid)

# RETIRED

In [ ]:
# Summarize buildings in parcels
inP['BLDG_I'] = 0
inP['BLDG_C'] = 0
inP['BLDG_I_G'] = 0
inP['BLDG_C_G'] = 0
for idx, row in inP.iterrows():
    # Summarize collected buildings
    potential_buildings = inB.loc[list(b_idx.intersection(row['geometry'].bounds))]
    m_bld = potential_buildings.loc[~potential_buildings.intersects(row['geometry'])]
    i_bld = potential_buildings.loc[potential_buildings.intersects(row['geometry'])]
    c_bld = potential_buildings.loc[potential_buildings['geometry'].apply(lambda x: row['geometry'].contains(x))]    
    inP.loc[idx, 'BLDG_I'] = i_bld.shape[0]
    inP.loc[idx, 'BLDG_C'] = c_bld.shape[0]
    
    # Summarize buildings in Google
    potential_buildings = inG.loc[list(g_idx.intersection(row['geometry'].bounds))]
    m_bld_g = potential_buildings.loc[~potential_buildings.intersects(row['geometry'])]
    i_bld_g = potential_buildings.loc[potential_buildings.intersects(row['geometry'])]
    c_bld_g = potential_buildings.loc[potential_buildings['geometry'].apply(lambda x: row['geometry'].contains(x))]    
    inP.loc[idx, 'BLDG_I_G'] = i_bld_g.shape[0]
    inP.loc[idx, 'BLDG_C_G'] = c_bld_g.shape[0]
    
    ''' # uncomment this section to stop the loop add a specific index in order to run plotting below
    if idx > 3:
        break
    '''

In [ ]:
m = folium.Map(location=[potential_buildings.unary_union.centroid.y, potential_buildings.unary_union.centroid.x], zoom_start=18)
try:
    folium.GeoJson(row['geometry'], 
             style_function=lambda x: {'fillOpacity':0.5, 'fillColor': 'red', 'color':'#520f0a'}).add_to(m)
except:
    pass
try:
    folium.GeoJson(data=m_bld.to_json(), 
              style_function=lambda x: {'fillOpacity':1,'fillColor': '#ebab15', 'stroke': False}).add_to(m)
except:
    pass
try:
    folium.GeoJson(i_bld.to_json(), 
               style_function=lambda x: {'fillOpacity':1, 'fillColor': '#6878ed', 'stroke': False}).add_to(m)
except:
    pass
try:
    folium.GeoJson(c_bld.to_json(), 
               style_function=lambda x: {'fillOpacity':1, 'fillColor': '#68ed76', 'stroke': False}).add_to(m)
except:
    pass

try:
    folium.GeoJson(data=potential_buildings.to_json(), 
              style_function=lambda x: {'fillOpacity':0,'fillColor': '#ebab15'}).add_to(m)
except:
    pass

m
